In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
import csv
def read_glove_vecs(glove_file ):
    with open(glove_file, 'r',encoding ='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [3]:
 def read_csv(filename):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y


In [4]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [53]:
X_train, Y_train = read_csv('Cmt_negative.csv')
#X_test, Y_test = read_csv('tesss.csv')
print(Y_train)

[ 4  0  0  0  9  9  1  0  0  0  4  2  0  3  0  2  7  1  1 10  1  0  0  9
  3  5  5  5  9 10  1  3  3  3  9  3  5  5  0  9  5  1  0  0  9  4  4  4
  3  4  3  4  2  0  1  2  5  3  3  3  3  4  3  4  0  0  4  0  5  6  6  9
  0  6  5  5  6  6  3  5  0  0  9  3  0  0  5  4  0 10  9  6  4  2  0  7
  1  4  0  2  6  0  9  0  5  2  9  9  4 10  4  3  5  2  0  0  2  0  4  2
  9  2  2  2  4  3  0  9  0  2  2  2]


In [54]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y


In [55]:
Y_oh_train = convert_to_one_hot(Y_train, C = 11)
#Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [56]:
 
Negative_Tags = {"0": "DegLien",    # :heart: prints a black instead of red heart depending on the font
                    "1": "DegEnv",
                    "2": "QVT",
                    "3": "OrgTrav",
                    "4": "PerTps",
                    "5": "Depers",
                    "6": "QuaProd",
                    "7": "RisSan",
                    "8": "NoIncV",
                    "9": "MatDis",
                    "10": "Confid "}

 
              

In [57]:
 Y_oh_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [42]:
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.python.framework import ops
ops.get_default_graph()
np.random.seed(1)

In [43]:
for idx, val in enumerate(["I", "like", "learning"]):
    print(idx,val)

0 I
1 like
2 learning


In [58]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =(X[i].lower()).split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+1
            
    ### END CODE HERE ###
    
    return X_indices

In [59]:
X1 = np.array(["waste of time finding an office with the right connection"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 10)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['waste of time finding an office with the right connection']
X1_indices =
 [[383684. 268046. 359889. 148433.  54273. 268206. 388711. 357266. 308561.
  107431.]]


In [60]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Step 1
    # Initialize the embedding matrix as a numpy array of zeros.
    # See instructions above to choose the correct shape.
    emb_matrix = np.zeros((vocab_len,emb_dim))
   # print(emb_matrix.shape)
    # Step 2
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    print("emb_matrix[1] :", emb_matrix[1,:])
    # Step 3
    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = Embedding(vocab_len,emb_dim)
    ### END CODE HERE ###

    # Step 4 (already done for you; please do not modify)
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [61]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])
 

emb_matrix[1] : [-0.58402   0.39031   0.65282  -0.3403    0.19493  -0.83489   0.11929
 -0.57291  -0.56844   0.72989  -0.56975   0.53436  -0.38034   0.22471
  0.98031  -0.2966    0.126     0.55222  -0.62737  -0.082242 -0.085359
  0.31515   0.96077   0.31986   0.87878  -1.5189   -1.7831    0.35639
  0.9674   -1.5497    2.335     0.8494   -1.2371    1.0623   -1.4267
 -0.49056   0.85465  -1.2878    0.60204  -0.35963   0.28586  -0.052162
 -0.50818  -0.63459   0.33889   0.28416  -0.2034   -1.2338    0.46715
  0.78858 ]
weights[0][1][3] = -0.3403


In [62]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    sentence_indices = Input(shape=input_shape, dtype=np.int32)
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer
    # (See additional hints in the instructions).
    eembeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(eembeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 5 units
    X = Dense(11, activation='softmax')(X)
    # Add a softmax activation
    X =  Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X)
    
    ### END CODE HERE ###
    
    return model

In [63]:
maxLen = 15
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

emb_matrix[1] : [-0.58402   0.39031   0.65282  -0.3403    0.19493  -0.83489   0.11929
 -0.57291  -0.56844   0.72989  -0.56975   0.53436  -0.38034   0.22471
  0.98031  -0.2966    0.126     0.55222  -0.62737  -0.082242 -0.085359
  0.31515   0.96077   0.31986   0.87878  -1.5189   -1.7831    0.35639
  0.9674   -1.5497    2.335     0.8494   -1.2371    1.0623   -1.4267
 -0.49056   0.85465  -1.2878    0.60204  -0.35963   0.28586  -0.052162
 -0.50818  -0.63459   0.33889   0.28416  -0.2034   -1.2338    0.46715
  0.78858 ]
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 15)]              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 15, 50)            20000050  
_________________________________________________________________
lstm_6 (LSTM)                (None, 15, 128)           91648     
__

In [64]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [65]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 11)

In [77]:
import tensorflow.keras
MonModelNegCmt = model.save('my_model_cmtneg')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model_cmtneg\assets


In [ ]:
model = tensorflow.keras.models.load_model('my_model_cmtneg')

In [72]:
model.fit(X_train_indices, Y_train_oh, epochs = 75 , batch_size = 29, shuffle=True)

Train on 132 samples
Epoch 1/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7665 - accuracy: 0.7803
Epoch 2/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7551 - accuracy: 0.7955
Epoch 3/75
132/132 [==============================] - 2s 14ms/sample - loss: 1.7452 - accuracy: 0.8030
Epoch 4/75
132/132 [==============================] - 2s 14ms/sample - loss: 1.7423 - accuracy: 0.8030
Epoch 5/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7420 - accuracy: 0.8030
Epoch 6/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7326 - accuracy: 0.8182
Epoch 7/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7578 - accuracy: 0.7879
Epoch 8/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7376 - accuracy: 0.8106
Epoch 9/75
132/132 [==============================] - 2s 13ms/sample - loss: 1.7715 - accuracy: 0.7803
Epoch 10/75
132/132 [==============================]

In [76]:
x_test = np.array(['stress to find a free office'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
#print(x_test[0] +' '+  label_to_emoji(np.argmax(Model_Emojify.predict(X_test_indices))))
print( model.predict(X_test_indices))
print(np.argmax(model.predict(X_test_indices)))

[[0.07872748 0.07872558 0.07876249 0.21201326 0.07873843 0.07939354
  0.07873225 0.07872698 0.07872625 0.07872775 0.07872599]]
3
